In [1]:
import numpy as np
import pandas as pd
lottorydf = pd.read_csv('lottory.csv')
lottorydf=lottorydf.sort_values(by=['期號'],ascending=True)
lottorydf=lottorydf.reset_index(drop=True)
lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6','預測2區']] = lottorydf[['前1','前2','前3','前4','前5','前6','后區']].shift(-1)
print(lottorydf.tail())
print(lottorydf.shape)

         期號  前1  前2  前3  前4  前5  前6  后區   預測1   預測2   預測3   預測4   預測5   預測6  \
416  110086  10  18  32  27   3  14   7  37.0  16.0  32.0  10.0  19.0  22.0   
417  110087  37  16  32  10  19  22   6  38.0  11.0   5.0  37.0   9.0  35.0   
418  110088  38  11   5  37   9  35   4   6.0  33.0  30.0  22.0   8.0  18.0   
419  110089   6  33  30  22   8  18   4  37.0   8.0   2.0  12.0   1.0  33.0   
420  110090  37   8   2  12   1  33   2   NaN   NaN   NaN   NaN   NaN   NaN   

     預測2區  
416   6.0  
417   4.0  
418   4.0  
419   2.0  
420   NaN  
(421, 15)


In [2]:
# print(lottorydf.iloc[:,8:14]/38)
lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6']]=lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6']]/38
lottorydf[['前1','前2','前3','前4','前5','前6']]=lottorydf[['前1','前2','前3','前4','前5','前6']]/38
lottorydf[['后區','預測2區']]=lottorydf[['后區','預測2區']]/8
print(lottorydf.tail())

         期號        前1        前2        前3        前4        前5        前6  \
416  110086  0.263158  0.473684  0.842105  0.710526  0.078947  0.368421   
417  110087  0.973684  0.421053  0.842105  0.263158  0.500000  0.578947   
418  110088  1.000000  0.289474  0.131579  0.973684  0.236842  0.921053   
419  110089  0.157895  0.868421  0.789474  0.578947  0.210526  0.473684   
420  110090  0.973684  0.210526  0.052632  0.315789  0.026316  0.868421   

        后區       預測1       預測2       預測3       預測4       預測5       預測6  預測2區  
416  0.875  0.973684  0.421053  0.842105  0.263158  0.500000  0.578947  0.75  
417  0.750  1.000000  0.289474  0.131579  0.973684  0.236842  0.921053  0.50  
418  0.500  0.157895  0.868421  0.789474  0.578947  0.210526  0.473684  0.50  
419  0.500  0.973684  0.210526  0.052632  0.315789  0.026316  0.868421  0.25  
420  0.250       NaN       NaN       NaN       NaN       NaN       NaN   NaN  


In [3]:
def preparedata(df, ref_day, predict_day):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,1:8]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day,8:15]))
    return np.array(X_train), np.array(Y_train)

In [6]:
X_train,Y_train= preparedata(lottorydf[:lottorydf.shape[0]-18],8,1)
print(np.ndim(X_train),X_train.shape)
# X_train=X_train.transpose(0,2,1)
# Y_train=Y_train.transpose(0,2,1)
# print(X_train.shape,len(X_train),Y_train.shape,len(Y_train),X_train.ndim)
X_test,Y_test = preparedata(lottorydf[lottorydf.shape[0]-18:],8,1)
# X_test=X_test.transpose(0,2,1)
# Y_test=Y_test.transpose(0,2,1)
# print(X_test.shape,len(X_test),Y_test.shape,len(Y_test),X_test.ndim)

3 (395, 8, 7)


In [8]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
import tensorflow as tf

In [17]:
model = Sequential()
model.add(LSTM(64,input_shape=(8,7),input_dim=3,activation='relu'))#, return_sequences=True, stateful=True))
model.add(Dense(7,activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                18432     
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 455       
Total params: 18,887
Trainable params: 18,887
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(X_train,Y_train, epochs=10,batch_size=1)


Epoch 1/10
395/395 [==============================] - 33s 78ms/step - loss: 0.0307 - accuracy: 0.8481
Epoch 2/10
395/395 [==============================] - 30s 77ms/step - loss: 0.0273 - accuracy: 0.8633
Epoch 3/10
395/395 [==============================] - 31s 78ms/step - loss: 0.0273 - accuracy: 0.8633
Epoch 4/10
395/395 [==============================] - 31s 78ms/step - loss: 0.0272 - accuracy: 0.8633
Epoch 5/10
395/395 [==============================] - 30s 77ms/step - loss: 0.0272 - accuracy: 0.8633
Epoch 6/10
395/395 [==============================] - 31s 79ms/step - loss: 0.0271 - accuracy: 0.8633
Epoch 7/10
395/395 [==============================] - 29s 73ms/step - loss: 0.0270 - accuracy: 0.8633
Epoch 8/10
395/395 [==============================] - 28s 71ms/step - loss: 0.0270 - accuracy: 0.8633
Epoch 9/10
395/395 [==============================] - 28s 72ms/step - loss: 0.0270 - accuracy: 0.8633
Epoch 10/10
395/395 [==============================] - 29s 72ms/step - loss: 0.027

In [25]:
model.evaluate(X_test,Y_test,batch_size=1)
predicts = model.predict(X_test,batch_size=1)
print(type(predicts))
print(predicts)
# print(model.predict_classes(X_test,batch_size=1))

10/10 [==============================] - 0s 36ms/step - loss: nan - accuracy: 0.3000
<class 'numpy.ndarray'>
[[0.15219116 0.27219602 0.41418913 0.5639573  0.716949   0.86934704
  0.5360265 ]
 [0.15414523 0.27381685 0.41348058 0.56925887 0.7222875  0.87151706
  0.52783775]
 [0.15410258 0.27208698 0.41504133 0.5686511  0.7192181  0.8726951
  0.52695274]
 [0.1485871  0.26885733 0.41357264 0.5663363  0.71460634 0.87283605
  0.53330445]
 [0.1468177  0.26620507 0.41371    0.5708226  0.7265054  0.8786912
  0.52549034]
 [0.14900531 0.26752552 0.41406044 0.57032615 0.72515965 0.8755789
  0.52713454]
 [0.1546687  0.2753992  0.41780546 0.56102747 0.71431667 0.86858803
  0.53496975]
 [0.1593162  0.28226268 0.41965088 0.5576731  0.7046958  0.8602445
  0.5334608 ]
 [0.16294536 0.2861005  0.41844898 0.55974704 0.7134077  0.86409944
  0.5306906 ]
 [0.155476   0.27615583 0.41747507 0.5612845  0.71797    0.8696358
  0.5349814 ]]


In [27]:
print(predicts.shape)
# a = predicts.transpose(0,2,1)
# new= a.squeeze()
new = pd.DataFrame(predicts)
new[[0,1,2,3,4,5]] = new[[0,1,2,3,4,5]]*38
new[[6]]=new[[6]]*8
new = new.astype(int)
new

(10, 7)


,0,1,2,3,4,5,6
0,5,10,15,21,27,33,4
1,5,10,15,21,27,33,4
2,5,10,15,21,27,33,4
3,5,10,15,21,27,33,4
4,5,10,15,21,27,33,4
5,5,10,15,21,27,33,4
6,5,10,15,21,27,33,4
7,6,10,15,21,26,32,4
8,6,10,15,21,27,32,4
9,5,10,15,21,27,33,4
